In [10]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import pandas as pd
import numpy as np

import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [12]:
# Lee la base de datos
df_nacional = pl.scan_csv(
    "../data/processed/egresos_procesados.csv",
    infer_schema_length=10000,
    null_values="Extranjero",
    dtypes={"EDAD_CALCULADA_ANO": pl.Float64}
)

In [13]:
# NeoNato es hasta 1 mes (sin incluirlo)
df_neonato = df_nacional.filter(pl.col("EDAD_CALCULADA_ANO") < 0.083333)

# Ninos es de 1 mes hasta 15 anios
df_ninos = df_nacional.filter(
    (pl.col("EDAD_CALCULADA_ANO") >= 0.083333) & (pl.col("EDAD_CALCULADA_ANO") < 15)
)

# Adolescente es de 15 anios hasta 18
df_adolescente = df_nacional.filter(
    (pl.col("EDAD_CALCULADA_ANO") >= 15) & (pl.col("EDAD_CALCULADA_ANO") < 18)
)

# El total del RDR seria desde los 0 anios hasta los 18
df_hasta_18 = df_nacional.filter((pl.col("EDAD_CALCULADA_ANO") < 18))

In [14]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "PERTENENCIA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

# Define los grupos que se quieren exportar
rankings_a_exportar = {
    "neonato": df_neonato,
    "ninos": df_ninos,
    "adolescente": df_adolescente,
    "hasta_18": df_hasta_18,
}

In [15]:
# Itera en los rangos etarios para exportar los rankings
for rango_etario, df_etario in rankings_a_exportar.items():
    print(rango_etario)
    metricas = build_features.obtener_metricas_egresos(df_etario, AGRUPACION).collect()
    metricas.write_csv(f"../data/interim/resumen_egresos_nacionales_{rango_etario}.csv")

neonato
ninos
adolescente
hasta_18
